## Landscape Energy Analysis

This notebook explores the energy landscape for different instances.

In [1]:
from qaoa_three_sat.simulation.landscape import build_landscape

# Import custom stuff
import os
import sys
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objs as go

os.chdir('..')

In [2]:
instance_filename = "sample_instance"
classical_opt_alg = "nelder-mead"
optimisation_opts = {
    "xtol": 0.001,
    "disp": True,
    "adaptive": True,
    "simplex_area_param": 0.1,
    "classical_opt_alg": "nelder-mead",
}

In [3]:
# Instance File
d_landscape = build_landscape(
    instance_filename=instance_filename,
    classical_opt_alg=classical_opt_alg,
    optimisation_opts=optimisation_opts,
    write_csv=False,
    
)

In [41]:
d_landscape[d_landscape['energy'] < -0.450930]

,alpha,beta,energy
844,-1.841593,-0.641593,-0.454615
907,-1.741593,-0.641593,-0.458990
908,-1.741593,-0.541593,-0.451779
970,-1.641593,-0.641593,-0.459250
971,-1.641593,-0.541593,-0.452859
1033,-1.541593,-0.641593,-0.455393
2998,1.558407,0.558407,-0.453752
2999,1.558407,0.658407,-0.454222
3061,1.658407,0.558407,-0.456257
3062,1.658407,0.658407,-0.457507


Create Grid for Contour and Surface Plots

In [48]:
alphas = d_landscape['alpha'].unique()
betas = d_landscape['beta'].unique()
energy = d_landscape['energy'].to_numpy()
energy = np.array(energy).reshape((len(alphas), len(betas)))

(63,)

In [55]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "surface"}, {"type":"contour"}]],
    subplot_titles=['3D Surface of Landscape', 'Contour Plot of Landscape'],

)
fig.add_trace(
    go.Surface(x=alphas, y=betas, z=energy, showscale=False, contours_z=dict(
                show=True,
                usecolormap=True,
                project_z=True,
            )),
    row=1, col=1
)
fig.add_trace(
    go.Contour(x=alphas, y=betas, z=energy, showscale=False),
    row=1, col=2
)
fig.update_layout(height=600, width=1100, title_text="Instance Energy Landscape")
fig.update_xaxes(title_text="beta", row=1, col=2)
fig.update_yaxes(title_text="alpha", row=1, col=2)
fig.show()

In [7]:
from qaoa_three_sat.simulation.simulate import simulate_circuit

In [32]:
alpha_trial = [1.67943456]
beta_trial = [0.61314886]
n_rounds = 1
track_optimiser = True

In [33]:
optimised_instance = simulate_circuit(
    instance_filename=instance_filename,
    classical_opt_alg=classical_opt_alg,
    optimisation_opts=optimisation_opts,
    alpha_trial=alpha_trial,
    beta_trial=beta_trial,
    n_rounds=n_rounds,
    track_optimiser=track_optimiser,
    disp=False
)

Optimization terminated successfully.
         Current function value: -0.460930
         Iterations: 16
         Function evaluations: 31
Optimal Sol:	 alpha:[1.67943456] beta:[0.61314886]


In [12]:
p_alpha = [i[0] for i in optimised_instance.d_alpha]
p_beta = [i[0] for i in optimised_instance.d_beta]
p_energy = [i for i in optimised_instance.d_energy]

# Validate lengths are ok
assert(len(p_alpha) == len(p_beta) and len(p_alpha) == len(p_energy))

In [62]:
fig = go.Figure(data=
    go.Contour(
        x=alphas,
        y=betas,
        z=energy,
        showscale=False,
        line = dict(width = 0)
    )
)

fig.add_trace(go.Scatter(y=p_alpha, x=p_beta, mode='lines',name='lines', line=dict(color="#ffffff")))
# Add END
fig.add_trace(
    go.Scatter(
        y=[p_alpha[-1]], 
        x=[p_beta[-1]], 
        mode='markers',
        name='markers', 
        line=dict(color="red")
    )
)

fig.update_xaxes(title_text="beta")
fig.update_yaxes(title_text="alpha")
fig.update_layout(height=600, width=1100, title_text="Instance Energy Landscape with Nelder-Mead Optimisation Path")
fig.show()

In [66]:
N = 5
rolling_mean_alpha = [np.mean(p_alpha[x:x+N]) for x in range(len(p_alpha)-N+1)]
rolling_mean_beta = [np.mean(p_beta[x:x+N]) for x in range(len(p_beta)-N+1)]
fig = go.Figure(data=
    go.Contour(
        x=alphas,
        y=betas,
        z=energy,
        showscale=False,
        line=dict(width=0)
    )
)
fig.add_trace(
    go.Scatter(
        y=rolling_mean_alpha, 
        x=rolling_mean_beta, 
        mode='lines',
        name='lines', 
        line=dict(color="#ffffff")
    )
)

# Add START 
fig.add_trace(
    go.Scatter(
        y=alpha_trial, 
        x=beta_trial, 
        mode='markers',
        name='markers', 
        line=dict(color="#00FF00")
    )
)
# Add END
fig.add_trace(
    go.Scatter(
        y=[p_alpha[-1]], 
        x=[p_beta[-1]], 
        mode='markers',
        name='markers', 
        line=dict(color="red")
    )
)
fig.update_xaxes(title_text="beta")
fig.update_yaxes(title_text="alpha")
fig.update_layout(height=600, width=1100, title_text="Instance Energy Landscape with Nelder-Mead Optimisation Path (smoothed)")
fig.show()